In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

## Check clip sizes

In [2]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'build') 

splits=C.split_files()

L=[x['t_seconds'] for x in splits]

In [3]:
min(L),max(L),sum(L)/len(L)

(0.135, 32.51, 3.6957927164784175)

## Train step 1: Bootstrap from pretrained model

In [4]:
from Cfg import Cfg
C = Cfg('NIST', 16000, 'amharic', 'build') 

In [5]:
from load_pretrained_amharic_model import load_pretrained_amharic_model
model = load_pretrained_amharic_model(C, 0)

[NeMo W 2020-10-16 19:18:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-16 19:18:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-16 19:18:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-16 19:18:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-16 19:18:18 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.TarredAudioToCharDataset'> is experimental,

[NeMo I 2020-10-16 19:18:19 features:241] PADDING: 16
[NeMo I 2020-10-16 19:18:19 features:258] STFT using torch
loaded save/nemo_amharic/amharic_20201015_220606_440181_epoch=199.ckpt


In [6]:
import pytorch_lightning as pl
import os, datetime

model_save_dir='save/nemo_amharic'

class ModelCheckpointAtEpochEnd(pl.callbacks.ModelCheckpoint):
    def on_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics['epoch'] = trainer.current_epoch
        trainer.checkpoint_callback.on_validation_end(trainer, pl_module)

pid=os.getpid()
dt=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

checkpoint_callback = ModelCheckpointAtEpochEnd(
    filepath=model_save_dir+'/amharic_'+f'{dt}_{pid}'+'_{epoch:02d}',
    verbose=True,
    save_top_k=-1,
    save_weights_only=False,
    period=1)

trainer = pl.Trainer(gpus=[0], max_epochs=1000, amp_level='O1', precision=16, checkpoint_callback=checkpoint_callback)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [7]:
from ruamel.yaml import YAML
from omegaconf import DictConfig
config_path = 'amharic_16000.yaml'
yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
train_manifest=f'{C.build_dir}/train_manifest.json'
test_manifest=f'{C.build_dir}/test_manifest.json'
params['model']['train_ds']['manifest_filepath'] = train_manifest
params['model']['validation_ds']['manifest_filepath'] = test_manifest
model.set_trainer(trainer)
model.setup_training_data(train_data_config=params['model']['train_ds'])
model.setup_validation_data(val_data_config=params['model']['validation_ds'])
model.setup_optimization(optim_config=DictConfig(params['model']['optim']))

[NeMo I 2020-10-16 19:18:47 collections:173] Dataset loaded with 9323 files totalling 9.58 hours
[NeMo I 2020-10-16 19:18:47 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-16 19:18:47 collections:173] Dataset loaded with 1036 files totalling 1.06 hours
[NeMo I 2020-10-16 19:18:47 collections:174] 0 files were filtered totalling 0.00 hours
[NeMo I 2020-10-16 19:18:47 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2020-10-16 19:18:47 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f837facb150>" 
    will be used during training (effective maximum steps = 776917) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 776917
    )


(Novograd (
 Parameter Group 0
     amsgrad: False
     betas: [0.8, 0.5]
     eps: 1e-08
     grad_averaging: False
     initial_lr: 0.0001
     lr: 0.0001
     weight_decay: 0.001
 ),
 {'scheduler': <nemo.core.optim.lr_scheduler.CosineAnnealing at 0x7f837facb150>,
  'interval': 'step',
  'frequency': 1,
  'monitor': 'val_loss',
  'reduce_on_plateau': False})

In [8]:
from reshuffle_samples import reshuffle_samples
reshuffle_samples(C)

In [ ]:
trainer.fit(model)

[NeMo I 2020-10-16 19:18:54 modelPT:572] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.0001
        weight_decay: 0.001
    )
[NeMo I 2020-10-16 19:18:54 lr_scheduler:545] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f8375880690>" 
    will be used during training (effective maximum steps = 776917) - 
    Parameters : 
    (warmup_steps: null
    warmup_ratio: null
    min_lr: 0.0
    last_epoch: -1
    max_steps: 776917
    )


[NeMo W 2020-10-16 19:18:54 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning:
    
    Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
    
    

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18 M  
2 | decoder           | ConvASRDecoder                    | 241 K 
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     


[NeMo I 2020-10-16 19:18:55 wer:148] 
    
[NeMo I 2020-10-16 19:18:55 wer:149] reference:ው አድጋለች ምላሰኛ ሆናለች በስመ አብ ምላሷን አችለውም
[NeMo I 2020-10-16 19:18:55 wer:150] decoded  :ው አድጋለች ምላሰኛ ሆናለች በስመ አብ ምላሷን አችለውም


[NeMo W 2020-10-16 19:18:56 nemo_logging:349] /home/catskills/anaconda3/envs/nemo/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning:
    
    Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
    
    


[NeMo I 2020-10-16 19:19:09 wer:148] 
    
[NeMo I 2020-10-16 19:19:09 wer:149] reference:ኢ ኢብራሂም የት
[NeMo I 2020-10-16 19:19:09 wer:150] decoded  :ኢ ኢብራሂም የት
[NeMo I 2020-10-16 19:19:23 wer:148] 
    
[NeMo I 2020-10-16 19:19:23 wer:149] reference:እ
[NeMo I 2020-10-16 19:19:23 wer:150] decoded  :እ
[NeMo I 2020-10-16 19:19:37 wer:148] 
    
[NeMo I 2020-10-16 19:19:37 wer:149] reference:ማን ማን ያስተናግድላቸዋል ታዲያ
[NeMo I 2020-10-16 19:19:37 wer:150] decoded  :ማን ማንናተያግ እንዝናቸዋ ላ
[NeMo I 2020-10-16 19:19:52 wer:148] 
    
[NeMo I 2020-10-16 19:19:52 wer:149] reference:እ
[NeMo I 2020-10-16 19:19:52 wer:150] decoded  :እ
[NeMo I 2020-10-16 19:20:06 wer:148] 
    
[NeMo I 2020-10-16 19:20:06 wer:149] reference:አዎ በቃ
[NeMo I 2020-10-16 19:20:06 wer:150] decoded  :አዎ በቃ
[NeMo I 2020-10-16 19:20:21 wer:148] 
    
[NeMo I 2020-10-16 19:20:21 wer:149] reference:ሰኞ እሮ እሮብ ዓርብ
[NeMo I 2020-10-16 19:20:21 wer:150] decoded  :ሰኛ እ እረክ አርብ
[NeMo I 2020-10-16 19:20:35 wer:148] 
    
[NeMo I 2020-10-16 19:20:3

...

[NeMo I 2020-10-16 22:58:56 wer:148] 
    
[NeMo I 2020-10-16 22:58:56 wer:149] reference:ኧረ ሰብሰብ ብሎ መቀመጡ ጥሩ ነው
[NeMo I 2020-10-16 22:58:56 wer:150] decoded  :ኧረ ሰብሰብ ብሎ መቀመጡ ጥሩ ነው
[NeMo I 2020-10-16 22:59:10 wer:148] 
    
[NeMo I 2020-10-16 22:59:10 wer:149] reference:ኦኬ እስከ እዛ ነዋ የምትሠሪው
[NeMo I 2020-10-16 22:59:10 wer:150] decoded  :ኦኬ እስከ እዛ ነዋ የምትሠሪው
[NeMo I 2020-10-16 22:59:25 wer:148] 
    
[NeMo I 2020-10-16 22:59:25 wer:149] reference:ተስማምቶሻላ ሁሉም ደህና ናቸው
[NeMo I 2020-10-16 22:59:25 wer:150] decoded  :ተስማምቶሻላ ሁሉም ደህና ናቸው
[NeMo I 2020-10-16 22:59:39 wer:148] 
    
[NeMo I 2020-10-16 22:59:39 wer:149] reference:ያው ምንም ከ ከእዛ ታይም በኋላ ከትምህርት ምልስ በኋላ እራሱ እሱ አብሮን አይውልም
[NeMo I 2020-10-16 22:59:39 wer:150] decoded  :ያው ምንም ከ ከእዛ ታይም በኋላኮ በትምህርት ምልስ በኋላ እራሱ እሱ አብሮን አይውልም
[NeMo I 2020-10-16 22:59:53 wer:148] 
    
[NeMo I 2020-10-16 22:59:53 wer:149] reference:ምን አዲስ ነገር አለ
[NeMo I 2020-10-16 22:59:53 wer:150] decoded  :ምን አዲስ ነገር አለ
[NeMo I 2020-10-16 23:00:08 wer:148] 
    
[NeMo I 202

[NeMo I 2020-10-16 23:02:27 wer:148] 
    
[NeMo I 2020-10-16 23:02:27 wer:149] reference:ው አድጋለች ምላሰኛ ሆናለች በስመ አብ ምላሷን አችለውም
[NeMo I 2020-10-16 23:02:27 wer:150] decoded  :ው አድጋለች ምላሰኛ ሆናለች በስመ አብ ላሷን አችለውም
[NeMo I 2020-10-16 23:02:27 wer:148] 
    
[NeMo I 2020-10-16 23:02:27 wer:149] reference:ሰላም ነው አንበጣ መንጋ በጣም አሁን ወረራ አድርጎ ነበር ያው የአንበጣውን መንጋ ለመከላከል ስንል
[NeMo I 2020-10-16 23:02:27 wer:150] decoded  :ሰላም ነው አንበጣ መንጋ በጣም አሁን ወረራ አድርጎ ነበር ያው የአንበጣውን መንጋ ለመከላከል ስንል
[NeMo I 2020-10-16 23:02:27 wer:148] 
    
[NeMo I 2020-10-16 23:02:27 wer:149] reference:አሁን ይሰማል እንደት ነህ አንተ
[NeMo I 2020-10-16 23:02:27 wer:150] decoded  :አሁን ይሰማል እንደት ነህ አንተ
[NeMo I 2020-10-16 23:02:27 wer:148] 
    
[NeMo I 2020-10-16 23:02:27 wer:149] reference:እሺ
[NeMo I 2020-10-16 23:02:27 wer:150] decoded  :እሺ
[NeMo I 2020-10-16 23:02:27 wer:148] 
    
[NeMo I 2020-10-16 23:02:27 wer:149] reference:ፊልም ቤት
[NeMo I 2020-10-16 23:02:27 wer:150] decoded  :ፊልም ቤት
[NeMo I 2020-10-16 23:02:27 wer:148] 
    
[NeMo I 2020-

[NeMo I 2020-10-16 23:02:30 wer:148] 
    
[NeMo I 2020-10-16 23:02:30 wer:149] reference:ኧረ ስለምን እንደ አባት አትሆንም እንደ አንተ ምንድን ነው እሺ
[NeMo I 2020-10-16 23:02:30 wer:150] decoded  :ኧረ ስለምን እንደ አባት አትሆንም እንደ አንተ ምንድን ነው እሺ
[NeMo I 2020-10-16 23:02:30 wer:148] 
    
[NeMo I 2020-10-16 23:02:30 wer:149] reference:እኔ እኔ ነገሩ አዲስ ነገር እስካሁን ያው
[NeMo I 2020-10-16 23:02:30 wer:150] decoded  :እኔ እኔ ነገሩ አዲስ ነገር እስካሁን ያው
[NeMo I 2020-10-16 23:02:30 wer:148] 
    
[NeMo I 2020-10-16 23:02:30 wer:149] reference:አዎ ሥራ ላይ እ
[NeMo I 2020-10-16 23:02:30 wer:150] decoded  :አዎ ሥራ ላይ እ
[NeMo I 2020-10-16 23:02:30 wer:148] 
    
[NeMo I 2020-10-16 23:02:30 wer:149] reference:ሄሎ
[NeMo I 2020-10-16 23:02:30 wer:150] decoded  :ሄሎ
[NeMo I 2020-10-16 23:02:30 wer:148] 
    
[NeMo I 2020-10-16 23:02:30 wer:149] reference:አይ ጥሩ እንደዚያ ከሆነ እኔም አላየውማ ካልወደድሽው
[NeMo I 2020-10-16 23:02:30 wer:150] decoded  :አይ ጥሩ ጥሩ እንደዚያ ከሆነ ኔማ ላየማ  ካለወደድሽም
[NeMo I 2020-10-16 23:02:30 wer:148] 
    
[NeMo I 2020-10-16 23:02:30 wer:149] re


Epoch 00057: saving model to /home/catskills/Desktop/openasr20/save/nemo_amharic/amharic_20201016_191843_62677_epoch=57.ckpt


[NeMo I 2020-10-16 23:02:49 wer:148] 
    
[NeMo I 2020-10-16 23:02:49 wer:149] reference:እሺ ነባ እንነግረዋለን እሺ
[NeMo I 2020-10-16 23:02:49 wer:150] decoded  :እሺ ነባ እንነግረዋለን እሺ
[NeMo I 2020-10-16 23:03:04 wer:148] 
    
[NeMo I 2020-10-16 23:03:04 wer:149] reference:አዎ
[NeMo I 2020-10-16 23:03:04 wer:150] decoded  :አዎ
[NeMo I 2020-10-16 23:03:18 wer:148] 
    
[NeMo I 2020-10-16 23:03:18 wer:149] reference:ትዕግስትስ ደህና ናት
[NeMo I 2020-10-16 23:03:18 wer:150] decoded  :ትግስትስ ደህና ናት
[NeMo I 2020-10-16 23:03:32 wer:148] 
    
[NeMo I 2020-10-16 23:03:32 wer:149] reference:እ በእውነት
[NeMo I 2020-10-16 23:03:32 wer:150] decoded  :እ በእውነት
[NeMo I 2020-10-16 23:03:46 wer:148] 
    
[NeMo I 2020-10-16 23:03:46 wer:149] reference:እ
[NeMo I 2020-10-16 23:03:46 wer:150] decoded  :እ
[NeMo I 2020-10-16 23:04:00 wer:148] 
    
[NeMo I 2020-10-16 23:04:00 wer:149] reference:እየሰራሽ ነው
[NeMo I 2020-10-16 23:04:00 wer:150] decoded  :እየሰራሽ ነው
[NeMo I 2020-10-16 23:04:15 wer:148] 
    
[NeMo I 2020-10-16 23:04:15 

## DEV translation

In [ ]:
from Cfg import Cfg
from glob import glob
from package_DEV import package_DEV
from load_pretrained_amharic_model import load_pretrained_amharic_model
version='113'
C = Cfg('NIST', 16000, 'amharic', 'dev', version)
model = load_pretrained_amharic_model(C, 0)
files=list(sorted(glob(f'{C.audio_split_dir}/*.wav')))
translations=model.transcribe(paths2audio_files=files, batch_size=32)